# Script to extract scattered raw GC data into one spreadsheet

## Imports and constants

In [1]:
import os
from pathlib import Path
import webbrowser
from functools import reduce
import random
import operator
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.offline as py
import plotly.graph_objs as go
import numpy as np
import sympy as sp
import qgrid as qg

from utils.plot_utils import scatterplot_from_df, boxplot_from_df 

sp.init_printing()
py.init_notebook_mode(connected=True)
qg.set_defaults(show_toolbar=True)
qg.enable()

defFile = "report00.CSV"
dataFile = "REPORT01.CSV"

requiredFiles = (defFile, dataFile)

## Define the input files and output dir

In [2]:
gcDir = "./data/GC/CO2H2/chromatograms"
gcPath = Path(gcDir)

vialCSV = "./data/GC/CO2H2/vials.csv"
vialCSVPath = Path(vialCSV)

sampleTimeCSV = "./data/sample-times.csv"
sampleTimePath = Path(sampleTimeCSV)

dilutionCSV = "./data/GC/CO2H2/dilutions.csv"
dilutionPath = Path(dilutionCSV)

outDir = "./intermediate/GC/CO2H2"
outPath = Path(outDir)

## Extract chromatograms information from nested folders into one dataframe

In [3]:
def isGcDir(files):
    return all(f in files for f in requiredFiles)

In [4]:
def getPeaksOfSample(subdir):
    subdirPath = Path(subdir)
        
    defs = pd.read_csv(subdirPath / defFile, header=None, encoding="ANSI", parse_dates=True, index_col=0)
    
    sampleName = defs.loc["Sample Name"].iloc[0]
    time = defs.loc["Injection Date"].iloc[0]
    
    dataCols = defs.loc["Column 1":, 1]
    dataCols = [col.strip() for col in dataCols]
    
    data = pd.read_csv(subdirPath / dataFile, header=None, encoding="ANSI", names=dataCols)
    data = data.reindex(["Sample Name", "Time", "Compound Name", "Retention Time", "Area"], axis="columns")
    
    data.loc[:,"Sample Name"] = sampleName
    data.loc[:,"Time"] = time
    
    return data


In [5]:
def extractPeaks():
    return (getPeaksOfSample(subdir) for subdir, dirs, files in os.walk(gcPath) if isGcDir(files))


In [6]:
peakTables = extractPeaks()
peaks = pd.concat(peakTables, axis="rows")
peaks.loc[:, "Time"] = pd.to_datetime(peaks.loc[:, "Time"])

In [7]:
peaks

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Have a look at all peaks with retention time and area

In [8]:
py.iplot(scatterplot_from_df(peaks, "Retention Time", "Area", seriesCol="Compound Name", xUnit="min"))

In [9]:
py.iplot(boxplot_from_df(peaks, "Area", seriesCol="Compound Name"))

## Correct false automatic associations of compounds IsobutOH and EtOH

In [10]:
def getAssociationWindow(df):
    return (
        df.loc[:, "Retention Time"].min(),
        df.loc[:, "Retention Time"].max()
    )

In [11]:
def genAssociateRow(compound, window):
    def associateRow(row):
        if(window[0] <= row.loc["Retention Time"] <= window[1]):
            row.loc["Compound Name"] = compound
            return row
        else:
            return row
        
    return associateRow

### Generate retention time windows from the min and max time recorded for EtOH and IsobutOH

In [12]:
compounds = ["IsobutOH", "EtOH"]

assocWins = pd.DataFrame(
    [getAssociationWindow(peaks.loc[peaks["Compound Name"] == c]) for c in compounds], 
    index=compounds, 
    columns=["min", "max"]
)

assocWins

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Manually adjust those windows to include more anassociated peaks and reduce stdev of IsobutOH standard

In [13]:
assocWins.loc["IsobutOH", "max"] = 4.3

assocWins

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Create a new table of peaks with adjusted compound association

In [14]:
newPeaks = peaks.copy()
        
newPeaks.loc[peaks["Compound Name"] == "?"] = newPeaks.loc[
    peaks["Compound Name"] == "?"
].apply(
    genAssociateRow("IsobutOH", assocWins.loc["IsobutOH"].values), 
    axis=1
)

newPeaks.loc[peaks["Compound Name"] == "?"] = newPeaks.loc[
    peaks["Compound Name"] == "?"
].apply(
    genAssociateRow("EtOH", assocWins.loc["EtOH"].values), 
    axis=1
)

agg = {
    "Retention Time": "mean",
    "Area": "sum"
}

newPeaks = newPeaks.groupby(["Sample Name", "Compound Name", "Time"]).agg(agg).reset_index()

### Have a look at the new peak data

In [15]:
newPeaks

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [16]:
py.iplot(scatterplot_from_df(newPeaks, "Retention Time", "Area", seriesCol="Compound Name"))

In [17]:
py.iplot(boxplot_from_df(newPeaks, "Area", seriesCol="Compound Name"))

## Process standards

In [18]:
stdPeaks = newPeaks.loc[newPeaks.loc[:,"Sample Name"].isin(["STD1", "STD2", "STD2", "STD4", "STD5"])]

### Have a look at STD peak area over time (should remain constant)

In [19]:
py.iplot(scatterplot_from_df(stdPeaks, "Time", "Area", seriesCol="Compound Name"))

### Get average area of all STD peaks by associated compound

In [20]:
stdMeans = stdPeaks.groupby("Compound Name").mean()

stdMeans

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Process sample data to get product mass concentrations

### Load association of vial name and sample name from input CSV

In [21]:
undilVials = pd.read_csv(vialCSVPath, index_col=0)

dilVials = undilVials.applymap(lambda x: "D"+str(x))

undilVials

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [22]:
undilVialAssoc = pd.melt(undilVials.reset_index(), id_vars=["Bottle"], value_vars=[*([str(i) for i in range(1, 13)])], var_name="Sample", value_name="Vial").set_index("Vial")

dilVialAssoc = pd.melt(dilVials.reset_index(), id_vars=["Bottle"], value_vars=[*([str(i) for i in range(1, 13)])], var_name="Sample", value_name="Vial").set_index("Vial")

vialAssoc = pd.concat([undilVialAssoc, dilVialAssoc])

vialAssoc.index = vialAssoc.index.astype(str)

### Extract all sample peaks into new table and include association with sampe and bottle name

In [23]:
def vialToSample(row):
    skey = row["Vial"]
    
    if(skey in vialAssoc.index):
        vialInfo = vialAssoc.loc[skey]
        return pd.concat([row, vialInfo])
    else:
        return row

In [24]:
samplePeaks = newPeaks.rename({"Sample Name": "Vial"}, axis="columns").apply(vialToSample, axis="columns").dropna()
samplePeaks = samplePeaks.loc[samplePeaks.loc[:,"Compound Name"].isin(["Acetate", "IsobutOH", "EtOH"])]

In [25]:
samplePeaks.loc[samplePeaks.loc[:,"Sample"] == "8"]

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Associate sample name with sampling time via input CSV

In [26]:
sampleTimes = pd.read_csv(sampleTimePath, index_col=0)
sampleTimes.index = sampleTimes.index.astype(str)

sampleTimes

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Add sample time column to sample peaks dataframe

In [27]:
def assocToBottleGroup(b):
    if b in ["A", "B", "C", "D"]:
        return "A-D"
    elif b in ["E", "F", "G", "H"]:
        return "E-H"
    else: return ""
    
def addSampleTime(row):
    if row["Sample"] in sampleTimes.index:
        row["Sample Time"] = sampleTimes.loc[sampleTimes.loc[:,"Bottle Set"] == assocToBottleGroup(row["Bottle"])].loc[row["Sample"]]["Time"]
    return row                                                                                                           
    
samplePeaks = samplePeaks.apply(addSampleTime, axis=1)

samplePeaks

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Add dilution factor to each row in sample peaks dataframe, again via input CSV

In [28]:
dilutions = pd.read_csv(dilutionPath, index_col=0)
dilutions.index = dilutions.index.astype(str)

dilutions

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [29]:
def addDilution(row):
    v = row["Vial"]
    s = row["Sample"]
    isDil = (v[0] == "D")
    
    if isDil and s in dilutions.index:
        dil = dilutions.loc[s]["Dilution"]
        if dil:
            row["Dilution"] = dil
        else:
            row["Dilution"] = 1
    else:
        row["Dilution"] = 1
        
    return row  

In [30]:
samplePeaks = samplePeaks.apply(addDilution, axis=1).sort_values(by=["Sample Time", "Bottle", "Compound Name"])
samplePeaks

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Define some physical constants

In [31]:
molMasses = {
    "EtOH": 46.068,
    "Acetate": 60.05,
    "IsobutOH": 74.122
}

conc = {
    "IsobutOH": 100*100/1100,
    "EtOH": 10,
    "Acetate": 10
}

### Calculate rF coefficients from mean IsobutOH area of STDs

In [32]:
ref = "IsobutOH"

calcRFs = lambda c1, a1, c2, a2: (c1/a1) / (c2/a2)

rFs = dict(
    [
        (name, calcRFs(conc[name], stdMeans.loc[name,"Area"], conc[ref], stdMeans.loc[ref,"Area"])) for name in conc
    ]
)

rFs

{'IsobutOH': 1.0, 'EtOH': 2.9918692775541, 'Acetate': 5.627748432445726}

### Perform calculation of mass concentrations from peak areas and rFs for all samples

In [33]:
isobutPeaks = samplePeaks.groupby(by="Compound Name").get_group("IsobutOH")

isobutPeaks = isobutPeaks.set_index(["Vial"])

In [34]:
def calculateConcentrations(row):
    compound = row["Compound Name"]
    isobutArea = isobutPeaks.loc[row["Vial"]]["Area"]
    row["Concentration"] = conc[ref] / float(isobutArea) * float(row["Area"]) * rFs[compound] / float(row["Dilution"])
    row["Mass Concentration"] = row["Concentration"] * molMasses[compound] / 1000 if compound in molMasses.keys() else float("nan")

    return row

In [35]:
samplePeaks = samplePeaks.apply(calculateConcentrations, axis=1).sort_values(by=["Sample Time", "Bottle", "Compound Name"])
samplePeaks

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Do some plotting with calculated mass concentrations

### Check how well concentrations calculated from diluted and from undiluted samples correlate

In [36]:
py.iplot(scatterplot_from_df(
    samplePeaks.loc[
        (samplePeaks.loc[:, "Bottle"] == "A") & (samplePeaks.loc[:, "Compound Name"] == "Acetate")
    ], 
    "Sample Time", 
    "Mass Concentration", 
    seriesCol="Dilution",
    nameCols=["Area"],
    connect=True
))

### Create a new dataframe with undiluted measurements replaced by diluted ones if available

In [37]:
samplePeaks.loc[:,"Is Diluted"] = pd.cut(samplePeaks.loc[:,"Dilution"], [0, 1, np.inf], right=False, labels=["diluted", "undiluted"])


In [38]:
def chooseDil(df):
    if "diluted" in df.loc[:,"Is Diluted"].values:
        return df.loc[df.loc[:,"Is Diluted"] == "diluted"]
    else:
        return df

dilSamplePeaks = pd.concat([chooseDil(df) for name, df in samplePeaks.groupby(by=["Sample", "Bottle", "Compound Name"])], axis=0)
dilSamplePeaks = dilSamplePeaks.sort_values(by=["Sample Time", "Bottle", "Compound Name"])

In [39]:
dilSamplePeaks

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Plot product mass concentrations of all bottles over time

In [40]:
py.iplot(scatterplot_from_df(
    samplePeaks.loc[
        (samplePeaks.loc[:, "Is Diluted"] == "undiluted") & (samplePeaks.loc[:, "Compound Name"] == "EtOH")
    ], 
    "Sample Time", 
    "Mass Concentration", 
    seriesCol="Bottle",
    nameCols=["Area"],
    connect=True
))

In [41]:
py.iplot(scatterplot_from_df(
    dilSamplePeaks.loc[
        (dilSamplePeaks.loc[:, "Compound Name"] == "Acetate")
    ], 
    "Sample Time", 
    "Mass Concentration", 
    seriesCol="Bottle",
    nameCols=["Vial", "Area"],
    connect=True
))

## Export peak and product mass concentration data

In [42]:
stdPeaks.rename({
    "Sample Name": "Vial"
}, axis="columns").sort_values(
    by="Vial"
).to_csv(outPath / "std-peaks.csv", index=False)

In [43]:
samplePeaks.to_csv(outPath / "sample-peaks.csv", index=False)

In [44]:
concentrations = samplePeaks.loc[samplePeaks["Compound Name"].isin(["Acetate", "EtOH"])].pivot_table(index=["Sample", "Bottle"], columns="Compound Name", values=["Mass Concentration"])

concentrations.columns = concentrations.columns.droplevel()
concentrations.index = concentrations.index.set_levels([concentrations.index.levels[0].astype(int), concentrations.index.levels[1]])
concentrations = concentrations.sort_index()

concentrations

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [45]:
concentrations.to_csv(outPath / "concentrations.csv")